In [4]:
import os
import sys
import numpy as np
import pandas as pd
import itertools
from tqdm import tqdm
import time

# Set up path to project root
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
sys.path.append(project_root)

# Custom modules
from src.problems.ising import relaxed_ising_energy, grad_relaxed_ising, ising_energy
from src.optimizers.sa import sa_continuous, sa_discrete
from src.optimizers.gd import gradient_descent
from src.utils.utils_experiments import bootstrap_experiment_ising, evaluate_continuous_results, evaluate_ising_results

# Output directory
results_dir = os.path.join(project_root, "results", "gridsearch")
os.makedirs(results_dir, exist_ok=True)

In [5]:
# Hyperparameter grids
sa_grid = {
    'T_init': [10, 50, 100],
    'alpha': [0.9, 0.95, 0.99, 0.995, 0.999],
    'step_size': [0.005, 0.01, 0.05, 0.1, 0.3, 0.5]
}

sa_grid_cont = {
    'T0': [10, 50, 100],
    'alpha': [0.9, 0.95, 0.99, 0.995, 0.999],
    'step_size': [0.005, 0.01, 0.05, 0.1, 0.3, 0.5]
}

gd_grid = {
    'lr': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.3, 0.5]
}

lattice_shape = (10, 10)
num_runs = 25
dim_ising = 10
x_inits_disc = [np.random.choice([-1, 1], size=lattice_shape) for _ in range(num_runs)]

In [6]:
def run_one_sa_discrete(T_init, alpha, num_runs=25):
    # Define known ground-truth state for evaluation
    best_state = np.ones(lattice_shape)

    # Set up initial spin configurations
    x_inits_disc = [
        np.random.choice([-1, 1], size=lattice_shape)
        for _ in range(num_runs)
    ]

    # Run bootstrap experiment
    result = bootstrap_experiment_ising(
        algorithm_function=sa_discrete,
        runs=num_runs,
        f=ising_energy,
        dim=dim_ising,
        x_inits=x_inits_disc,
        is_discrete=True,
        best_state=best_state,
        hamming_threshold=0,
        T_init=T_init,
        alpha=alpha,
        lattice_size=lattice_shape,
        max_iter=20000,
        tol=1e-6,
        f_star=-200.0
    )

    stats = result["stats"]
    
    return {
        "algorithm": "SA_discrete",
        "T_init": T_init,
        "alpha": alpha,
        "mean": stats.get("mean"),
        "best": stats.get("best"),
        "worst": stats.get("worst"),
        "std": stats.get("std"),
        "rmse" : stats.get("rmse"),
        "mse" : stats.get("rmse"),
        "mean_runtime": stats.get("mean_runtime"),
        "hamming_best_to_gt": stats.get("hamming_best_to_gt")
    }

def grid_search_sa_discrete(grid, num_runs=25):
    combos = list(itertools.product(grid["T_init"], grid["alpha"]))
    results = []

    for idx, (T_init, alpha) in enumerate(tqdm(combos), 1):
        print(f"[{idx}/{len(combos)}] T_init = {T_init}, alpha = {alpha}")
        res = run_one_sa_discrete(T_init, alpha, num_runs)
        results.append(res)

    return pd.DataFrame(results)

In [7]:
def run_one_sa_continuous(T0, alpha, step_size, num_runs=25):

    # define known ground-truth state for evaluation
    best_state = np.ones(lattice_shape)

    x_inits_cont = [
        np.random.uniform(low=-1.0, high=1.0, size=lattice_shape)
        for _ in range(num_runs)
    ]

    result = bootstrap_experiment_ising(
        algorithm_function=sa_continuous,
        runs=num_runs,
        f=relaxed_ising_energy,
        grad_f=None,
        dim=dim_ising,
        x_inits=x_inits_cont,
        is_discrete=False,
        T0=T0,
        alpha=alpha,
        step_size=step_size,
        max_iter=20000,
        tol=1e-6,
        f_star=-200.0,
        init_range = (-1,1)
    )

    stats = result["stats"]

    return {
        "algorithm": "SA_continuous",
        "T0": T0,
        "alpha": alpha,
        "step_size": step_size,
        "mean": stats.get("mean"),
        "best": stats.get("best"),
        "worst": stats.get("worst"),
        "std": stats.get("std"),
        "rmse": stats.get("rmse"),
        "mse": stats.get("mse"),
        "mean_runtime": stats.get("mean_runtime"),
        "mean_distance_to_x_star": stats.get("mean_distance_to_x_star")
    }

def grid_search_sa_continuous(grid, num_runs=25):
    combos = list(itertools.product(grid["T0"], grid["alpha"], grid["step_size"]))
    results = []

    for idx, (T0, alpha, step_size) in enumerate(tqdm(combos), 1):
        print(f"[{idx}/{len(combos)}] T0 = {T0}, alpha = {alpha}, step_size = {step_size}")
        res = run_one_sa_continuous(T0, alpha, step_size, num_runs)
        results.append(res)

    return pd.DataFrame(results)

In [8]:
def run_one_gd_ising_continuous(lr, num_runs=25):
    best_state = np.ones(lattice_shape)

    x_inits_cont = [
        np.random.uniform(low=-1.0, high=1.0, size=lattice_shape)
        for _ in range(num_runs)
    ]
    x_inits_flat = [x.flatten() for x in x_inits_cont]

    # Wrap f and grad_f to handle flattened input
    relaxed_f = lambda x: relaxed_ising_energy(x.reshape(lattice_shape))
    grad_wrapped = lambda x: grad_relaxed_ising(x.reshape(lattice_shape)).flatten()

    result = bootstrap_experiment_ising(
        algorithm_function=gradient_descent,
        runs=num_runs,
        f=relaxed_f,
        grad_f=grad_wrapped,
        dim=dim_ising,
        x_inits=x_inits_flat,
        is_discrete=False,
        lr=lr,
        max_iter=20000,
        tol=1e-6,
        f_star=-200.0,
        init_range=(-1, 1)
    )

    stats = result["stats"]
    return {
        "algorithm": "GD_continuous",
        "lr": lr,
        "mean": stats.get("mean"),
        "best": stats.get("best"),
        "worst": stats.get("worst"),
        "std": stats.get("std"),
        "rmse": stats.get("rmse"),
        "mse": stats.get("mse"),
        "mean_runtime": stats.get("mean_runtime"),
        "mean_distance_to_x_star": stats.get("mean_distance_to_x_star")
    }


def grid_search_gd_ising_continuous(grid, num_runs=25):
    results = []

    for idx, lr in enumerate(tqdm(grid["lr"]), 1):
        print(f"[{idx}/{len(grid['lr'])}] lr = {lr}")
        res = run_one_gd_ising_continuous(lr, num_runs)
        results.append(res)

    return pd.DataFrame(results)



# Running actual Gridsearch

In [9]:
# print("\nRunning SA-discrete on ising_energy")
# df_sa_discrete = grid_search_sa_discrete(sa_grid, num_runs=25)
# df_sa_discrete.to_csv(os.path.join(results_dir, "gridsearch_sa_discrete_ising_new.csv"), index=False)

In [10]:
# print("\nRunning SA-continuous on relaxed_ising")
# df_sa_continuous = grid_search_sa_continuous(sa_grid_cont, num_runs=25)
# df_sa_continuous.to_csv(os.path.join(results_dir, "gridsearch_sa_continuous_ising_new.csv"), index=False)

In [ ]:
print("\nRunning GD on relaxed_ising_energy")
df_gd = grid_search_gd_ising_continuous(gd_grid, num_runs=25)
df_gd.to_csv(os.path.join(results_dir, "gridsearch_gd_continuous_ising_new.csv"), index=False)


Running GD on relaxed_ising_energy


  0%|          | 0/7 [00:00<?, ?it/s]

[1/7] lr = 0.0001
[Ising Relaxed] Run 1/25


Clip range : (-1, 1)
[Ising Relaxed] Run 2/25
Clip range : (-1, 1)
[Ising Relaxed] Run 3/25
Clip range : (-1, 1)
[Ising Relaxed] Run 4/25
Clip range : (-1, 1)
[Ising Relaxed] Run 5/25
Clip range : (-1, 1)
[Ising Relaxed] Run 6/25
Clip range : (-1, 1)
[Ising Relaxed] Run 7/25
Clip range : (-1, 1)
[Ising Relaxed] Run 8/25
Clip range : (-1, 1)
[Ising Relaxed] Run 9/25
Clip range : (-1, 1)
[Ising Relaxed] Run 10/25
Clip range : (-1, 1)
[Ising Relaxed] Run 11/25
Clip range : (-1, 1)
[Ising Relaxed] Run 12/25
Clip range : (-1, 1)
[Ising Relaxed] Run 13/25
Clip range : (-1, 1)
[Ising Relaxed] Run 14/25
Clip range : (-1, 1)
[Ising Relaxed] Run 15/25
Clip range : (-1, 1)
[Ising Relaxed] Run 16/25
Clip range : (-1, 1)
[Ising Relaxed] Run 17/25
Clip range : (-1, 1)
[Ising Relaxed] Run 18/25
Clip range : (-1, 1)
[Ising Relaxed] Run 19/25
Clip range : (-1, 1)
[Ising Relaxed] Run 20/25
Clip range : (-1, 1)
[Ising Relaxed] Run 21/25
Clip range : (-1, 1)
[Ising Relaxed] Run 22/25
Clip range : (-1, 1)


 14%|█▍        | 1/7 [01:34<09:24, 94.10s/it]

[2/7] lr = 0.001
[Ising Relaxed] Run 1/25
Clip range : (-1, 1)
[Ising Relaxed] Run 2/25
Clip range : (-1, 1)
[Ising Relaxed] Run 3/25
Clip range : (-1, 1)
[Ising Relaxed] Run 4/25
Clip range : (-1, 1)
[Ising Relaxed] Run 5/25
Clip range : (-1, 1)
[Ising Relaxed] Run 6/25
Clip range : (-1, 1)
[Ising Relaxed] Run 7/25
Clip range : (-1, 1)
[Ising Relaxed] Run 8/25
Clip range : (-1, 1)
[Ising Relaxed] Run 9/25
Clip range : (-1, 1)
[Ising Relaxed] Run 10/25
Clip range : (-1, 1)
[Ising Relaxed] Run 11/25
Clip range : (-1, 1)
[Ising Relaxed] Run 12/25
Clip range : (-1, 1)
[Ising Relaxed] Run 13/25
Clip range : (-1, 1)
[Ising Relaxed] Run 14/25
Clip range : (-1, 1)
[Ising Relaxed] Run 15/25
Clip range : (-1, 1)
[Ising Relaxed] Run 16/25
Clip range : (-1, 1)
[Ising Relaxed] Run 17/25
Clip range : (-1, 1)
[Ising Relaxed] Run 18/25
Clip range : (-1, 1)
[Ising Relaxed] Run 19/25
Clip range : (-1, 1)
[Ising Relaxed] Run 20/25
Clip range : (-1, 1)
[Ising Relaxed] Run 21/25
Clip range : (-1, 1)
[Isin

 29%|██▊       | 2/7 [01:45<03:48, 45.70s/it]

[3/7] lr = 0.01
[Ising Relaxed] Run 1/25
Clip range : (-1, 1)
[Ising Relaxed] Run 2/25
Clip range : (-1, 1)
[Ising Relaxed] Run 3/25
Clip range : (-1, 1)
[Ising Relaxed] Run 4/25
Clip range : (-1, 1)
[Ising Relaxed] Run 5/25
Clip range : (-1, 1)
[Ising Relaxed] Run 6/25
Clip range : (-1, 1)
[Ising Relaxed] Run 7/25
Clip range : (-1, 1)
[Ising Relaxed] Run 8/25
Clip range : (-1, 1)
[Ising Relaxed] Run 9/25
Clip range : (-1, 1)
[Ising Relaxed] Run 10/25
Clip range : (-1, 1)
[Ising Relaxed] Run 11/25
Clip range : (-1, 1)
[Ising Relaxed] Run 12/25
Clip range : (-1, 1)
[Ising Relaxed] Run 13/25
Clip range : (-1, 1)
[Ising Relaxed] Run 14/25
Clip range : (-1, 1)
[Ising Relaxed] Run 15/25
Clip range : (-1, 1)
[Ising Relaxed] Run 16/25
Clip range : (-1, 1)
[Ising Relaxed] Run 17/25
Clip range : (-1, 1)
[Ising Relaxed] Run 18/25
Clip range : (-1, 1)
[Ising Relaxed] Run 19/25
Clip range : (-1, 1)
[Ising Relaxed] Run 20/25
Clip range : (-1, 1)
[Ising Relaxed] Run 21/25
Clip range : (-1, 1)
[Ising

 43%|████▎     | 3/7 [01:47<01:41, 25.46s/it]

Clip range : (-1, 1)
[4/7] lr = 0.05
[Ising Relaxed] Run 1/25
Clip range : (-1, 1)
[Ising Relaxed] Run 2/25
Clip range : (-1, 1)
[Ising Relaxed] Run 3/25
Clip range : (-1, 1)
[Ising Relaxed] Run 4/25
Clip range : (-1, 1)
[Ising Relaxed] Run 5/25
Clip range : (-1, 1)
[Ising Relaxed] Run 6/25
Clip range : (-1, 1)
[Ising Relaxed] Run 7/25
Clip range : (-1, 1)
[Ising Relaxed] Run 8/25
Clip range : (-1, 1)
[Ising Relaxed] Run 9/25
Clip range : (-1, 1)
[Ising Relaxed] Run 10/25
Clip range : (-1, 1)
[Ising Relaxed] Run 11/25
Clip range : (-1, 1)
[Ising Relaxed] Run 12/25
Clip range : (-1, 1)
[Ising Relaxed] Run 13/25
Clip range : (-1, 1)
[Ising Relaxed] Run 14/25
Clip range : (-1, 1)
[Ising Relaxed] Run 15/25
Clip range : (-1, 1)
[Ising Relaxed] Run 16/25
Clip range : (-1, 1)
[Ising Relaxed] Run 17/25
Clip range : (-1, 1)
[Ising Relaxed] Run 18/25
Clip range : (-1, 1)
[Ising Relaxed] Run 19/25
Clip range : (-1, 1)
[Ising Relaxed] Run 20/25
Clip range : (-1, 1)
[Ising Relaxed] Run 21/25
Clip r